In [1]:
from dataset import MPII_dataset
from models import HourglassModel
import tensorflow as tf
import keras.backend as k
import argparse

Using TensorFlow backend.


In [2]:
IMAGES_DIR = 'images'
ANNOTATIONS_JSON_FILE = 'annotations/annotations.json'
INPUT_SHAPE = (256, 256, 3)
OUTPUT_SHAPE = (64, 64, 16)
BATCH_SIZE = 8
STACKS_NUM = 2

In [3]:
mpii_dataset = MPII_dataset(
        images_dir=IMAGES_DIR,
        annots_json_filename=ANNOTATIONS_JSON_FILE,
        input_shape=INPUT_SHAPE,
        output_shape=OUTPUT_SHAPE,
        type='train'
    )

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0

In [5]:
k.tensorflow_backend.set_session(tf.Session(config=config))

In [6]:
hg_model = HourglassModel(
    dataset=mpii_dataset,
    batch_size=BATCH_SIZE,
    stacks_num=STACKS_NUM,
)

In [7]:
hg_model.build()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
front_module_conv2d_7x7 (Conv2D (None, 128, 128, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 64) 256         front_module_conv2d_7x7[0][0]    
__________________________________________________________________________________________________
front_residual_1_conv_1x1_first (None, 128, 128, 64) 4160        batch_normalization_1[0][0]      
_____________________________________

In [8]:
hg_model.train()

A
Instructions for updating:
Use tf.cast instead.
Epoch 1/1


KeyboardInterrupt: 